In [17]:
import chipwhisperer as cw
import time

# Initialize ChipWhisperer scope and XMEGA target
scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial2)

# Set scope parameters for clock glitching
scope.clock.clkgen_freq = 7.37e6  # Set clock frequency (e.g., 7.37 MHz)
scope.io.tio1 = "serial_tx"  # Set serial output
scope.io.tio2 = "serial_rx"  # Set serial input
scope.glitch.clk_src = "clkgen"  # Use internal clock generator for glitches
scope.glitch.output = "enable_only"  # Enable only glitches

# Set initial glitch parameters
scope.glitch.offset = -48  # Glitch offset (timing)
scope.glitch.width = 0  # Glitch width (duration)
scope.glitch.trigger_src = "ext_single"  # Use external trigger
scope.glitch.repeat = 1  # Number of glitch repeats
scope.adc.timeout = 0.1  # Timeout for ADC capture

# XMEGA target-specific initialization
target.baud = 38400  # Set baud rate for communication

# Function to reset the target
def reset_target():
    scope.io.nrst = "low"
    time.sleep(0.1)
    scope.io.nrst = "high"
    time.sleep(0.1)
    target.flush()  # Flush any remaining data

# Function to run a single glitch attempt
def run_glitch():
    reset_target()  # Reset the target before each attempt
    
    scope.arm()  # Arm the scope for capturing glitches
    
    target.simpleserial_write("g", bytearray([]))  # Send a command to the target to provoke a response
    
    ret = scope.capture()  # Capture the glitch
    
    if ret:
        print('Timeout - no trigger')
    else:
        try:
            response = target.simpleserial_read('r', 4)  # Read the response
            print(f"Response: {response}")
        except Exception as e:
            print(f"Error: {e}")

# Main glitching loop
num_attempts = 10  # Number of glitch attempts
for i in range(num_attempts):
    print(f"Attempt {i+1}")
    run_glitch()

# Cleanup
scope.dis()
target.dis()


(ChipWhisperer NAEUSB WARNING|File naeusb.py:802) Your firmware (0.11) is outdated - latest is 0.65 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


USBErrorIO: LIBUSB_ERROR_IO [-1]

In [18]:
import chipwhisperer as cw
import time
import numpy as np

# Initialize ChipWhisperer scope and XMEGA target
scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial2)

# Set scope parameters for clock glitching
scope.clock.clkgen_freq = 7.37e6  # Set clock frequency (e.g., 7.37 MHz)
scope.io.tio1 = "serial_tx"  # Set serial output
scope.io.tio2 = "serial_rx"  # Set serial input
scope.glitch.clk_src = "clkgen"  # Use internal clock generator for glitches
scope.glitch.output = "enable_only"  # Enable only glitches
scope.glitch.trigger_src = "ext_single"  # Use external trigger
scope.glitch.repeat = 1  # Number of glitch repeats
scope.adc.timeout = 0.1  # Timeout for ADC capture

# XMEGA target-specific initialization
target.baud = 38400  # Set baud rate for communication

# Function to reset the target
def reset_target():
    scope.io.nrst = "low"
    time.sleep(0.1)
    scope.io.nrst = "high"
    time.sleep(0.1)
    target.flush()  # Flush any remaining data

# Function to run a single glitch attempt
def run_glitch(offset, width):
    scope.glitch.offset = offset
    scope.glitch.width = width
    
    reset_target()  # Reset the target before each attempt
    
    scope.arm()  # Arm the scope for capturing glitches
    
    target.simpleserial_write("g", bytearray([]))  # Send a command to the target to provoke a response
    
    ret = scope.capture()  # Capture the glitch
    
    if ret:
        print(f'Timeout - no trigger (Offset: {offset}, Width: {width})')
        return False, None
    else:
        try:
            response = target.simpleserial_read('r', 4)  # Read the response
            return True, response
        except Exception as e:
            print(f"Error: {e} (Offset: {offset}, Width: {width})")
            return False, None

# Main glitching loop to find optimal parameters
optimal_params = None
for offset in range(-48, 49, 4):  # Adjust range and step size as needed
    for width in range(0, 49, 4):  # Adjust range and step size as needed
        success, response = run_glitch(offset, width)
        if success:
            print(f"Successful glitch (Offset: {offset}, Width: {width}, Response: {response})")
            optimal_params = (offset, width)
            break
    if optimal_params:
        break

if optimal_params:
    print(f"Optimal parameters found: Offset = {optimal_params[0]}, Width = {optimal_params[1]}")
else:
    print("No successful glitch found within the tested parameters.")

# Cleanup
scope.dis()
target.dis()
import chipwhisperer as cw
import time
import numpy as np

# Initialize ChipWhisperer scope and XMEGA target
scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial2)

# Set scope parameters for clock glitching
scope.clock.clkgen_freq = 7.37e6  # Set clock frequency (e.g., 7.37 MHz)
scope.io.tio1 = "serial_tx"  # Set serial output
scope.io.tio2 = "serial_rx"  # Set serial input
scope.glitch.clk_src = "clkgen"  # Use internal clock generator for glitches
scope.glitch.output = "enable_only"  # Enable only glitches
scope.glitch.trigger_src = "ext_single"  # Use external trigger
scope.glitch.repeat = 1  # Number of glitch repeats
scope.adc.timeout = 0.1  # Timeout for ADC capture

# XMEGA target-specific initialization
target.baud = 38400  # Set baud rate for communication

# Function to reset the target
def reset_target():
    scope.io.nrst = "low"
    time.sleep(0.1)
    scope.io.nrst = "high"
    time.sleep(0.1)
    target.flush()  # Flush any remaining data

# Function to run a single glitch attempt
def run_glitch(offset, width):
    scope.glitch.offset = offset
    scope.glitch.width = width
    
    reset_target()  # Reset the target before each attempt
    
    scope.arm()  # Arm the scope for capturing glitches
    
    target.simpleserial_write("g", bytearray([]))  # Send a command to the target to provoke a response
    
    ret = scope.capture()  # Capture the glitch
    
    if ret:
        print(f'Timeout - no trigger (Offset: {offset}, Width: {width})')
        return False, None
    else:
        try:
            response = target.simpleserial_read('r', 4)  # Read the response
            return True, response
        except Exception as e:
            print(f"Error: {e} (Offset: {offset}, Width: {width})")
            return False, None

# Main glitching loop to find optimal parameters
optimal_params = None
for offset in range(-48, 49, 4):  # Adjust range and step size as needed
    for width in range(0, 49, 4):  # Adjust range and step size as needed
        success, response = run_glitch(offset, width)
        if success:
            print(f"Successful glitch (Offset: {offset}, Width: {width}, Response: {response})")
            optimal_params = (offset, width)
            break
    if optimal_params:
        break

if optimal_params:
    print(f"Optimal parameters found: Offset = {optimal_params[0]}, Width = {optimal_params[1]}")
else:
    print("No successful glitch found within the tested parameters.")

# Cleanup
scope.dis()
target.dis()


(ChipWhisperer NAEUSB WARNING|File naeusb.py:802) Your firmware (0.11) is outdated - latest is 0.65 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information
(ChipWhisperer Glitch WARNING|File ChipWhispererGlitch.py:786) Partial reconfiguration for width = 0 may not work
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
(ChipWhisperer NAEUSB WARNING|File naeusb.py:802) Your firmware (0.11) is outdated - latest is 0.65 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information
(ChipWhisperer Glitch WARNING|File ChipWhispererGlitch.py:786) Partial reconfiguration for width = 0 may not work


Successful glitch (Offset: -48, Width: 0, Response: None)
Optimal parameters found: Offset = -48, Width = 0


(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 


Successful glitch (Offset: -48, Width: 0, Response: None)
Optimal parameters found: Offset = -48, Width = 0


In [21]:
import chipwhisperer as cw
import time
import numpy as np

# Initialize ChipWhisperer scope and XMEGA target
scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial2)

# Set scope parameters for clock glitching
scope.clock.clkgen_freq = 7.37e6  # Set clock frequency (e.g., 7.37 MHz)
scope.io.tio1 = "serial_tx"  # Set serial output
scope.io.tio2 = "serial_rx"  # Set serial input
scope.glitch.clk_src = "clkgen"  # Use internal clock generator for glitches
scope.glitch.output = "enable_only"  # Enable only glitches

# Set optimal glitch parameters found previously
optimal_offset = -48  # Example value, replace with the actual optimal value
optimal_width = 0  # Example value, replace with the actual optimal value
scope.glitch.offset = optimal_offset
scope.glitch.width = optimal_width
scope.glitch.trigger_src = "ext_single"  # Use external trigger
scope.glitch.repeat = 1  # Number of glitch repeats
scope.adc.timeout = 0.1  # Timeout for ADC capture

# XMEGA target-specific initialization
target.baud = 38400  # Set baud rate for communication

# Function to reset the target
def reset_target():
    scope.io.nrst = "low"
    time.sleep(0.1)
    scope.io.nrst = "high"
    time.sleep(0.1)
    target.flush()  # Flush any remaining data

# Function to run a single glitch attempt and capture the for loop results
def run_glitch():
    reset_target()  # Reset the target before each attempt
    
    scope.arm()  # Arm the scope for capturing glitches
    
    target.simpleserial_write("p", bytearray([]))  # Send a command to the target to provoke a response
    
    ret = scope.capture()  # Capture the glitch
    
    if ret:
        print('Timeout - no trigger')
    else:
        try:
            response = []
            for _ in range(100):
                data = target.simpleserial_read('r', 1, glitch_timeout=10)
                response.append(data)
            print(f"Response: {response}")
            return response
        except Exception as e:
            print(f"Error: {e}")
            return None

# Main glitching loop
num_attempts = 10  # Number of glitch attempts
for i in range(num_attempts):
    print(f"Attempt {i+1}")
    response = run_glitch()
    if response:
        # Analyze the response to determine if the glitch was successful
        if len(response) != 100 or any(r[0] != i for i, r in enumerate(response)):
            print(f"Glitch successful on attempt {i+1}: {response}")
            break

# Cleanup
scope.dis()
target.dis()


(ChipWhisperer NAEUSB WARNING|File naeusb.py:802) Your firmware (0.11) is outdated - latest is 0.65 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information
(ChipWhisperer Glitch WARNING|File ChipWhispererGlitch.py:786) Partial reconfiguration for width = 0 may not work


Attempt 1
Error: simpleserial_read() got an unexpected keyword argument 'glitch_timeout'
Attempt 2
Error: simpleserial_read() got an unexpected keyword argument 'glitch_timeout'
Attempt 3
Error: simpleserial_read() got an unexpected keyword argument 'glitch_timeout'
Attempt 4
Error: simpleserial_read() got an unexpected keyword argument 'glitch_timeout'
Attempt 5
Error: simpleserial_read() got an unexpected keyword argument 'glitch_timeout'
Attempt 6
Error: simpleserial_read() got an unexpected keyword argument 'glitch_timeout'
Attempt 7
Error: simpleserial_read() got an unexpected keyword argument 'glitch_timeout'
Attempt 8
Error: simpleserial_read() got an unexpected keyword argument 'glitch_timeout'
Attempt 9
Error: simpleserial_read() got an unexpected keyword argument 'glitch_timeout'
Attempt 10
Error: simpleserial_read() got an unexpected keyword argument 'glitch_timeout'
